# Movement of actors

<div class="admonition note">
    <p class="admonition-title">In progress</p>
    <p>This document is a work in progress if you see any errors, or exclusions or have any problems, please <a href="https://github.com/absespy/ABSESpy/issues">get in touch with us</a>.</p>
</div>

ABSESpy provides methods such as `move_to`, `put_on`, `put_on_layer` at the agent level to reposition agents across the grid or grids layers that support the model. Model agents' ability to explore their environment is crucial in the context of agent-based simulation and complex systems.

The following constitutes a brief overview of the methods available to agents to move around the grid and layers. There will not be any underlying logic underlying agents' movement to keep it simple. Instead, they will pick a random position from its neighborhood to move to. 

This examples illustrates movement of agents across a single layer world. The same methods work for multi-layered worlds, but careful attention is advised because it would involve using the methods with some caution according to the context of the action (i.e movement between layers as opposed to within a layer).

The model consists of a model with a single layer and a single type of agent. Every model must define a population and a grid or nature component for the agents to be able to move around. 

Particular attention must be paid to the sections that define the location of the agents. Initially, each agent must be placed somewhere on the grid, for which the `put_on` and `put_on_layer` method are useful. The choice for this case is arbitrary and consist of placing each agent the same initial position.
``` python
class MockModel(MainModel):
    ...
    def setup(self):
        ...
        # place each actor on the layer, at the same position (0, 0)
        for actor in self.actors:
            actor.put_on_layer(layer=self.nature.major_layer, pos=(0, 0))
```
More importantly, each step, each agent will choose a new location to move. In this case, the agent will choose its next position randomly from its neighborhood. The `move_to` method is used to move the agent to the new position. The `move_to` method should be the default approach to relocating agents across the grid while the `put_on` and `put_on_layer` methods are best used to place agents on the grid or layers for the first time and other special cases.
``` python
class MovingActor(Actor):
    ...
    def step(self):
        self.move_to(self.random.choice(self.layer.get_neighborhood(...)))
```

In [3]:
from abses import Actor, MainModel
from abses.datacollection import DataCollector


class MovingActor(Actor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def step(self):
        # pick a random, neighboring cell or position to move to
        neighborhood = self.layer.get_neighborhood(self.pos, moore=True)

        # pick a position randomly to move to
        new_pos = self.random.choice(neighborhood)
        self.move_to(new_pos)


class MockModel(MainModel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def setup(self):
        # @TODO: create another layer, a rasterized one, to illustrate movement

        self.agents.create(MovingActor, 10)

        self.nature.create_module(how="from_resolution", shape=(10, 10))

        # place each actor on the layer, at the same position (0, 0)
        for actor in self.actors:
            actor.put_on_layer(layer=self.nature.major_layer, pos=(0, 0))

        self.datacollector = DataCollector(
            self, agent_reporters={"position": "pos"}
        )

    def step(self):
        # step agents
        self.actors.trigger("step")

        # collect data
        self.datacollector.collect()

Once the code for the model's object is written, me may move on to the next step of the process, which is to instantiate the model object and run the model. From there, we want to retrieve and visually inspect the model's output.

After running successfully, we observe the effect that our simple rule for the movement of one agent had on the model. 

In [2]:
model = MockModel(seed=42, parameters={"time": {"end": 10}})

model.run_model()

model.datacollector.get_agent_vars_dataframe().unstack(level=-1)

[2024-01-14 11:47:39][human          ] | Initializing a new Human Module...
[2024-01-14 11:47:39][nature         ] | Initializing a new Base Nature module...
[2024-01-14 11:47:39][main           ] | Setting up MockModel...
[2024-01-14 11:47:39][container      ] | Created 10 actors of breed MovingActor
[2024-01-14 11:47:39][nature         ] | Initializing a new Model Layer...
[2024-01-14 11:47:39][nature         ] | Using rioxarray version: 0.13.4
[2024-01-14 11:47:39][datacollection ] | DataCollector component initialized.
[2024-01-14 11:47:39][main           ] | Ending MockModel


position                                                          \
AgentID       1       2       3       4       5       6       7       8    
Step                                                                       
0         (1, 1)  (0, 1)  (0, 1)  (0, 1)  (1, 1)  (0, 1)  (1, 1)  (0, 1)   
1         (0, 0)  (1, 2)  (0, 2)  (1, 1)  (0, 1)  (0, 0)  (0, 0)  (1, 2)   
2         (1, 1)  (2, 2)  (0, 3)  (1, 0)  (0, 2)  (1, 0)  (1, 0)  (2, 2)   
3         (0, 1)  (3, 1)  (1, 3)  (0, 1)  (1, 1)  (0, 0)  (0, 1)  (2, 3)   
4         (1, 2)  (3, 2)  (0, 3)  (0, 0)  (2, 0)  (0, 1)  (1, 1)  (2, 4)   
5         (0, 2)  (3, 1)  (1, 2)  (1, 1)  (3, 1)  (0, 0)  (1, 0)  (3, 3)   
6         (0, 3)  (3, 2)  (2, 1)  (1, 2)  (2, 1)  (0, 1)  (2, 0)  (4, 3)   
7         (1, 2)  (2, 1)  (2, 0)  (1, 1)  (1, 1)  (1, 1)  (1, 1)  (3, 4)   
8         (2, 1)  (2, 0)  (3, 1)  (2, 0)  (1, 0)  (0, 1)  (2, 1)  (2, 3)   
9         (3, 1)  (3, 1)  (3, 2)  (3, 0)  (2, 0)  (0, 2)  (1, 2)  (3, 3)   

                         
AgentID      9       10  
Step                     
0        (0, 1)  (1, 0)  
1        (0, 2)  (0, 0)  
2        (0, 1)  (1, 1)  
3        (0, 0)  (2, 0)  
4        (1, 0)  (3, 1)  
5        (0, 1)  (2, 1)  
6        (1, 0)  (3, 0)  
7        (1, 1)  (4, 0)  
8        (1, 0)  (4, 1)  
9        (0, 1)  (4, 2)